# Test

In [1]:
#First we need to load the model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_name = "model/checkpoints/grpo_llama3b/epoch_0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="cuda:0")
model.eval()


/home/jqi/miniconda3/envs/llm_marl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.33it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

In [12]:
system_prompt = '''A conversation between User and Assistant. The user asks a question, and the Assistant solves it.
    The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
    The reasoning process and answer are enclosed within <think></think> and <answer></answer> tags, respectively,
    i.e., <think>reasoning process here</think> <answer>answer here</answer>.'''

question = "User: Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?  Assistant: <think>"

input_prompt = system_prompt + question

input_ids = tokenizer.encode(input_prompt, return_tensors="pt").to("cuda:0")

output = model.generate(input_ids, max_new_tokens=512, do_sample=True, top_p=0.95, top_k=60, temperature=0.1, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(output[0], skip_special_tokens=True))

A conversation between User and Assistant. The user asks a question, and the Assistant solves it.
    The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
    The reasoning process and answer are enclosed within <think></think> and <answer></answer> tags, respectively,
    i.e., <think>reasoning process here</think> <answer>answer here</answer>.User: Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?  Assistant: <think>One glass costs $5
So, the second glass costs $5 * 0.6 = $<<5*0.6=3>>3
So, the total cost of the second glass is $3
So, the total cost of the glasses is $5 * 16 = $<<5*16=80>>80
And the cost of the second glass is $3 * 16 = $<<3*16=48>>48
So, the total cost of the second glasses is $48
So, the total cost of the glasses is $80 - 48 = $<<80-48=32>>32
</think> <answe

# Unit Test of the functions

In [1]:
import sys
import time
from functools import partial
from typing import Any, Dict, List, Optional, Union
from warnings import warn

import torch
from omegaconf import DictConfig, ListConfig
from torch import nn
from torch.distributed import destroy_process_group, init_process_group
from torch.optim import Optimizer
from torchdata.stateful_dataloader import StatefulDataLoader
from torchdata.stateful_dataloader.sampler import StatefulDistributedSampler
from torchtune import config, generation, modules, rlhf, training, utils
from torchtune.config._utils import _get_component_from_path
from torchtune.datasets import ConcatDataset
from torchtune.dev.grpo.generation import generate
from torchtune.dev.grpo.rewards import batch_shaped_correctness_reward
from torchtune.dev.grpo.types import GRPOStats, GRPOTrajectory
from torchtune.modules import local_kv_cache
from torchtune.recipe_interfaces import FTRecipeInterface
from torchtune.training import disable_dropout, DummyProfiler, PROFILER_KEY
from torchtune.training.lr_schedulers import get_lr
from tqdm import tqdm


/home/jqi/miniconda3/envs/llm_marl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torchtune.models.llama3_2 import llama3_2_3b
from torchtune.modules import local_kv_cache
import torch

model = llama3_2_3b()

In [3]:
from torchtune.models.llama3 import Llama3Tokenizer
model_name = "model/llama3B_gsm8k_sft_part0/epoch_0/original/tokenizer.model"
tokenizer = Llama3Tokenizer(model_name)

In [4]:
model.eval()
model.to("cuda:0")

TransformerDecoder(
  (tok_embeddings): Embedding(128256, 3072)
  (layers): ModuleList(
    (0-27): 28 x TransformerSelfAttentionLayer(
      (attn): MultiHeadAttention(
        (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
        (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (output_proj): Linear(in_features=3072, out_features=3072, bias=False)
        (pos_embeddings): Llama3ScaledRoPE()
      )
      (mlp): FeedForward(
        (w1): Linear(in_features=3072, out_features=8192, bias=False)
        (w2): Linear(in_features=8192, out_features=3072, bias=False)
        (w3): Linear(in_features=3072, out_features=8192, bias=False)
        (activation): SiLU()
      )
      (sa_norm): RMSNorm()
      (mlp_norm): RMSNorm()
      (sa_scale): Identity()
      (mlp_scale): Identity()
    )
  )
  (norm): RMSNorm()
)

In [5]:
#First we need to load the model and tokenizer
import torch

# from torchtune.models.llama3_2 import llama3_2_3b
from torchtune.modules import local_kv_cache
import torch

# model = llama3_2_3b()



system_prompt = '''A conversation between User and Assistant. The user asks a question, and the Assistant solves it.
    The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
    The reasoning process and answer are enclosed within <think></think> and <answer></answer> tags, respectively,
    i.e., <think>reasoning process here</think> <answer>answer here</answer>.'''

question = "User: Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?  Assistant: <think>"

input_prompt = system_prompt + question

input_ids = torch.tensor(tokenizer.encode(input_prompt)).to("cuda:0")
#make sure input_ids matches batch_size, context_length 
input_ids = input_ids.unsqueeze(0)

answers = ["64"]

max_generated_tokens = 512

temperature = 0.1

top_k = 60

rng = torch.Generator(device="cuda:0").manual_seed(0)


import torch
import torch.nn as nn

def generate_trajectory(
    model,
    ref_model,
    tokenizer,
    input_ids,
    answers,
    grpo_samples=8,
    max_generated_tokens=128,
    temperature=1.0,
    top_k=None,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    dtype=torch.float32,
    rng=None,
) -> GRPOTrajectory:
    """
    Generates a trajectory given the current policy model, the reference policy model, the reward function,
    and batch of inputs. This is done over the following steps:

    1: Generate responses, and logits corresponding to the responses using the current policy,
        generating (query, response) pairs.
    2. Estimate logprobs of the generated responses using the current policy.
    3. Compute rewards and successes for the generated responses.
    4. Estimate advantages using GRPO.
    5. Replace any tokens in the response after the first stop token (usually EOS token) with padding,
        producing truncated responses.

    Args:
        model (torch.nn.Module): Current policy model
        ref_model (torch.nn.Module): Reference policy model
        tokenizer (Any): Tokenizer with pad_id and stop_tokens attributes
        input_ids (torch.Tensor): tensor of input token IDs with shape [b, seq_length]
        answers (List[str]): list of answers corresponding to the input_ids
        grpo_samples (int): Number of samples per input. Default 8
        max_generated_tokens (int): Maximum number of tokens to generate. Default 128
        temperature (float): Sampling temperature. Default 1.0
        top_k (Optional[int]): Top-k sampling parameter. Default None
        device (torch.device): Device to run on. Default CUDA if available else CPU
        dtype (torch.dtype): Data type to use. Default float32
        rng (Optional[torch.Generator]): Random number generator. Default None

    Returns:
        Trajectory: An instance of :class:`~torchtune.rlhf.GRPOTrajectory` comprising
            the current trajectory.
    """
    batch_size, context_length = input_ids.shape

    batch_input_ids = input_ids[:, None, :].expand(-1, grpo_samples, -1)  # [B, G, L]
    batch_input_ids = batch_input_ids.reshape(batch_size * grpo_samples, -1)

    # step 1: generate responses, and logits corresponding to the responses using the current policy

    with local_kv_cache(
        model=model,
        batch_size=batch_size * grpo_samples,
        device=device,
        dtype=dtype,
        decoder_max_seq_len=context_length + max_generated_tokens,
    ):
        query_responses, _ = generate(  # [B x G, L], [B x G, L, V]
            model=model,
            prompt=batch_input_ids,
            max_generated_tokens=max_generated_tokens,
            temperature=temperature,
            top_k=top_k,
            pad_id=tokenizer.pad_id,
            rng=rng,
            stop_tokens=tokenizer.stop_tokens,
            return_logits=False,
        )

    print("generate done")
    print(query_responses)
    print(query_responses.shape)
    # torch.distributed.barrier()
    # training._distributed.recursive_reshard(model)
    # torch.cuda.empty_cache()

    responses = query_responses[:, context_length:].clone()
    query_response_padding_masks = query_responses != tokenizer.pad_id

    # step 1.1 create attention masks and position IDs for any padding tokens in inputs, used for future forward passes
    masks = generation.get_causal_mask_from_padding_mask(
        query_response_padding_masks
    )
    position_ids = generation.get_position_ids_from_padding_mask(
        query_response_padding_masks
    )

    del query_response_padding_masks

    logits = model(query_responses, input_pos=position_ids, mask=masks)

    # step 2. estimate logprobs of the responses using the current policy
    logits = logits[:, context_length - 1 :]
    logprobs = rlhf.batched_logits_to_logprobs(logits, responses, temperature)

    del logits
    torch.cuda.empty_cache()

    # step 2.1 estimate logprobs of the responses using the reference policy
    ref_logits = ref_model(
        query_responses, input_pos=position_ids, mask=masks
    )
    ref_logits = rlhf.truncate_sequence_for_logprobs(ref_logits, context_length)
    ref_logprobs = rlhf.batched_logits_to_logprobs(
        ref_logits, responses, temperature
    )

    del ref_logits
    torch.cuda.empty_cache()

    # step 4. replace any tokens in the responses after the first stop token (usually EOS token) with padding
    # resulting in truncated responses
    response_padding_masks = torch.ones_like(responses)
    # (
    #     response_padding_masks,
    #     responses,
    # ) = rlhf.truncate_sequence_at_first_stop_token(  # [B x G, L]
    #     responses, tokenizer.stop_tokens, tokenizer.pad_id
    # )

    # responses :: [B x G, L]
    responses = responses.reshape(batch_size, grpo_samples, -1)  # [B, G, L]

    rewards, successes = batch_shaped_correctness_reward(
        tokenizer, responses, answers
    )  # [B, G]
    rewards = rewards.to(device)
    successes = successes.to(device)

    advantages = (rewards - rewards.mean(1, keepdim=True)) / (
        rewards.std(1, keepdim=True) + 1e-4
    )
    advantages = advantages.reshape(batch_size * grpo_samples)  # flatten

    del responses
    torch.cuda.empty_cache()

    seq_lens = training.get_unmasked_sequence_lengths(response_padding_masks)

    # step 6. mask out all the invalid values in the trajectory due to padding tokens
    logprobs[response_padding_masks] = 1.0
    ref_logprobs[response_padding_masks] = 1.0

    return GRPOTrajectory(
        query_responses=query_responses,
        logprobs=logprobs,
        ref_logprobs=ref_logprobs,
        rewards=rewards.reshape(batch_size * grpo_samples),
        successes=successes.reshape(batch_size * grpo_samples),
        advantages=advantages,
        masks=masks,
        position_ids=position_ids,
        response_padding_masks=response_padding_masks,
        seq_lens=seq_lens,
    )

In [6]:
trajectory = generate_trajectory(
    model,
    model,
    tokenizer,
    input_ids,
    answers,
)

generate done
tensor([[128000,     32,  10652,  ...,     29, 128001, 128001],
        [128000,     32,  10652,  ...,     29, 128001, 128001],
        [128000,     32,  10652,  ...,     29, 128001, 128001],
        ...,
        [128000,     32,  10652,  ...,     29, 128001, 128001],
        [128000,     32,  10652,  ...,     29, 128001, 128001],
        [128000,     32,  10652,  ...,     29, 128001, 128001]],
       device='cuda:0')
torch.Size([8, 151])


TypeError: isin() received an invalid combination of arguments - got (Tensor, list), but expected one of:
 * (Tensor elements, Tensor test_elements, *, bool assume_unique = False, bool invert = False, Tensor out = None)
 * (Number element, Tensor test_elements, *, bool assume_unique = False, bool invert = False, Tensor out = None)
 * (Tensor elements, Number test_element, *, bool assume_unique = False, bool invert = False, Tensor out = None)


In [14]:
from torchtune.dev.selfplay.rewards import extract_tags_eval, shaped_correctness_reward_eval

text = '''To calculate the average walking pace, which is the distance traveled divided by the time taken, we need to calculate the distance traveled in hours. Given that he walks 10 km every 2 hours, the pace can be calculated as 10 km / 2 hours = 5 km per hour.
</think> <answer>5</answer>
User: A beekeeper has 150 beehives and needs to fill two different jars filled with honey. The larger jar can hold 11 times as much honey as the smaller jar. The larger jar is filled 50% while the smaller jar is filled to 25% of their respective capacities. How many jellybeans' worth of honey are there in the jars altogether? Assume a given conversion rate where each jellybean holds 5 grams, and 1 liter of honey is equivalent to 3527 jellybeans.
Assistant: <think>First, we need to calculate the space occupied by the honey. Given that the larger jar is filled 50% and can hold 11 times as much honey as the smaller jar, the space occupied by the honey in the larger jar is 50% * 11 jars = 55% of the smaller jar. So, the space occupied by the honey in the smaller jar is 25% since the larger jar takes up 55%. 
In order to calculate the number of jellybeans in the jars, we need to convert the space occupied to liters. From there, it is a simple matter of converting the honey to jellybeans using the conversionThe Assistant evaluate the problem and the solution. The assistant first evaluates and find out where the solution is wrong. Then the assistant provides the verification. The evaluation process and verification are enclosed within <evaluate></evaluate> and <verify></verify> tags, respectively, i.e., <evaluate>evaluation process here</evaluate> <verify>verification here</verify>. Verification is limited to 'correct' or 'wrong'. Assistant:<evaluate>Based on the scenario described, we first need to convert 2.5 hours movie duration to minutes, which is 2.5 * 60 = 150 minutes. Then, we need to calculate the number of times John uses the bathroom during the movie by dividing the total movie duration (150 minutes) by the interval (50 minutes per bathroom). This gives us an answer of 150 minutes / 50 minutes per bathroom = 3 bathroom usages.
</evaluate> <verify>correct</verify>
Assistant: The user asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think></think> and <answer></answer> tags, respectively.'''

tags = extract_tags_eval("<think>" + text.replace("<<", "").replace(">>", ""))
tags

ParseError: mismatched tag: line 7, column 289 (<string>)

In [12]:
shaped_correctness_reward_eval("15", text)

(110.0, 0.0, 1, 1.0, 0.0, 0.0)

In [5]:
verification = tags["verify"][-1].lower() if len(tags["verify"]) > 2 else ""
verification

'wrong'

In [17]:
if verification == "correct" or verification == "wrong":
    print("yes")

yes


(10.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [ ]:
evaluated_trajectory.query_responses A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think></think> and <answer></answer> tags, respectively, i.e., <think>reasoning process here</think> <answer>answer here</answer>. User: Herbert is 10 years younger than Kris. If Kris is 24 years old now, how old will Herbert be next year?. Assistant: <think>First, let's find Herbert's current age by subtracting 10 years from Kris' current age. Herbert is 24 - 10 = 14 years old. Next, we want to know how old Herbert will be next year. We take Herbert's current age and add one year, so Herbert will be 14 + 1 = 15 years old next year.</think> <answer>15</answer>The Assistant evaluate the problem and the solution. The assistant first evaluates and find out where the solution is wrong. Then the assistant provides the verification. The evaluation process and verification are enclosed within <evaluate></evaluate> and <verify></verify> tags, respectively, i.e., <evaluate>evaluation process here</evaluate> <verify>verification here</verify>. Verification is limited to 'correct' or 'wrong'. Assistant:<evaluate>According to reasoning, Herbert will be 15 years old next year. The assistant thinks that the instructions state correctly that Herbert will be 15 years old next year.</evaluate> <verify>wrong</verify><|endoftext|>!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [44]:
result = extract_tags_eval("<think>" + text.replace("<<", "").replace(">>", ""))
result["answer"][0]

'15'

In [36]:
from torchtune import config, generation, modules, rlhf, training, utils
import torch
responses = torch.tensor([[],[] ])
(
        response_padding_masks,
        responses,
    ) = rlhf.truncate_sequence_at_first_stop_token(  # [B x G, L]
        responses, torch.tensor(128012), torch.tensor(128011)
    )
print(response_padding_masks)
print(responses)

tensor([], size=(2, 0), dtype=torch.bool)
tensor([], size=(2, 0))


In [37]:
#[rank1]: IndexError: argmax(): Expected reduction dim 1 to have non-zero size.
seq_lens = training.get_unmasked_sequence_lengths(response_padding_masks)
print(seq_lens)

IndexError: argmax(): Expected reduction dim 1 to have non-zero size.

In [42]:
l = torch.tensor([[128012, 128011]]).shape[1]

torch.tensor([[128012, 128011]])[:,l:]

tensor([], size=(1, 0), dtype=torch.int64)

In [4]:
answer = '$69'
if '69' == answer:
    print("yes")
elif '69' in answer:
    print("no")

no
